## Installs


In [1]:
%pip install penaltyblog

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install aesara

In [4]:
import os
import pandas as pd

## Workout


In [41]:
current_gameweek = pb.fpl.get_current_gameweek()
current_gameweek

13

In [42]:
info = pb.fpl.get_gameweek_info()
info.head()

,id,name,deadline_time,release_time,average_entry_score,finished,data_checked,highest_scoring_entry,deadline_time_epoch,deadline_time_game_offset,...,h2h_ko_matches_created,ranked_count,chip_plays,most_selected,most_transferred_in,top_element,top_element_info,transfers_made,most_captained,most_vice_captained
0,1,Gameweek 1,2024-08-16 17:30:00+00:00,None,57,True,True,3546234.0,1723829400,0,...,False,8597356,"[{'chip_name': 'bboost', 'num_played': 144974}...",401.0,27.0,328.0,"{'id': 328, 'points': 14}",0,351.0,351.0
1,2,Gameweek 2,2024-08-24 10:00:00+00:00,None,69,True,True,9442126.0,1724493600,0,...,True,9543353,"[{'chip_name': 'bboost', 'num_played': 118782}...",401.0,594.0,177.0,"{'id': 177, 'points': 20}",13364453,351.0,401.0
2,3,Gameweek 3,2024-08-31 10:00:00+00:00,None,64,True,True,430195.0,1725098400,0,...,True,9912161,"[{'chip_name': 'bboost', 'num_played': 87334},...",351.0,177.0,328.0,"{'id': 328, 'points': 17}",23723836,351.0,328.0
3,4,Gameweek 4,2024-09-14 10:00:00+00:00,None,51,True,True,3560750.0,1726308000,0,...,True,10178753,"[{'chip_name': 'bboost', 'num_played': 83637},...",351.0,129.0,185.0,"{'id': 185, 'points': 15}",25930392,351.0,328.0
4,5,Gameweek 5,2024-09-21 10:00:00+00:00,None,58,True,True,1773336.0,1726912800,0,...,True,10279610,"[{'chip_name': 'bboost', 'num_played': 66447},...",351.0,58.0,180.0,"{'id': 180, 'points': 16}",13036058,351.0,351.0


In [52]:
mappings = pb.fpl.get_player_id_mappings()
mappings

,first_name,second_name,web_name,id
0,Fábio,Ferreira Vieira,Fábio Vieira,1
1,Gabriel,Fernando de Jesus,G.Jesus,2
2,Gabriel,dos Santos Magalhães,Gabriel,3
3,Kai,Havertz,Havertz,4
4,Karl,Hein,Hein,5
...,...,...,...,...
680,Bastien,Meupiyou,Meupiyou,637
681,André,Trindade da Costa Neto,André,642
682,Carlos Roberto,Forbs Borges,Forbs,655
683,Alfie,Pond,Pond,664


In [51]:
players = pb.fpl.get_player_data()
players

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,position,position_short,team_id,team_name,team_name_short
0,0.0,0.0,438098,0,0,-1,1,0,3,0.0,...,306,632,280,0.00,0.00,Midfielder,MID,1,Arsenal,ARS
1,NaN,NaN,85955,0,0,-3,3,0,3,1.0,...,188,343,117,1.36,0.45,Midfielder,MID,1,Arsenal,ARS
2,100.0,100.0,444145,0,0,-1,1,1,3,3.5,...,17,92,29,1.06,0.40,Midfielder,MID,1,Arsenal,ARS
3,NaN,NaN,499175,0,0,0,0,0,3,2.8,...,120,275,90,0.00,0.00,Midfielder,MID,1,Arsenal,ARS
4,100.0,100.0,184029,1,-1,-2,2,0,3,4.2,...,21,75,22,1.06,0.64,Midfielder,MID,1,Arsenal,ARS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
680,NaN,NaN,517179,0,0,-1,1,0,2,0.2,...,119,314,116,0.00,0.00,Defender,DEF,20,Wolves,WOL
681,100.0,100.0,101982,0,0,-1,1,0,1,0.0,...,27,282,41,1.00,0.00,Goalkeeper,GKP,20,Wolves,WOL
682,NaN,NaN,79602,0,0,0,0,0,1,0.0,...,36,163,24,0.00,0.00,Goalkeeper,GKP,20,Wolves,WOL
683,NaN,NaN,149065,0,0,-2,2,0,1,2.5,...,26,142,21,1.00,0.14,Goalkeeper,GKP,20,Wolves,WOL


In [50]:
zincs = mappings.query("web_name == 'G.Jesus'")
zincs_id = zincs["id"].iloc[0]
players.query("id == @zincs_id")

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,...,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90,position,position_short,team_id,team_name,team_name_short
14,100.0,100.0,205651,0,0,-2,2,0,4,1.8,...,48,211,36,0.85,0.43,Forward,FWD,1,Arsenal,ARS


In [53]:
players.query("position_short == 'GKP'")[
    ["id", "first_name", "second_name", "position", "total_points", "minutes"]
].sort_values("total_points", ascending=False).head()

,id,first_name,second_name,position,total_points,minutes
475,383,André,Onana,Goalkeeper,56,1080
285,235,Jordan,Pickford,Goalkeeper,52,1080
541,443,Matz,Sels,Goalkeeper,52,1080
30,15,David,Raya Martin,Goalkeeper,48,1080
507,413,Nick,Pope,Goalkeeper,45,1080


In [54]:
history = pb.fpl.get_player_history(zincs_id)
history

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,minutes,...,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out
0,2,2,20,0,True,2024-08-17 14:00:00+00:00,2.0,0.0,1,5,...,0,0.00,0.00,0.00,0.15,7.0,0,199810,0,0
1,2,11,2,0,False,2024-08-24 16:30:00+00:00,0.0,2.0,2,0,...,0,0.00,0.00,0.00,0.00,6.9,-53975,176166,12170,66145
2,2,21,5,0,True,2024-08-31 11:30:00+00:00,1.0,1.0,3,0,...,0,0.00,0.00,0.00,0.00,6.8,-72583,106691,3238,75821
3,2,39,18,1,False,2024-09-15 13:00:00+00:00,0.0,1.0,4,10,...,0,0.00,0.00,0.00,0.13,6.8,-27513,82577,1997,29510
4,2,47,13,0,False,2024-09-22 15:30:00+00:00,2.0,2.0,5,3,...,0,0.00,0.00,0.00,0.69,6.8,1642,89599,7682,6040
5,2,51,11,1,True,2024-09-28 14:00:00+00:00,4.0,2.0,6,1,...,0,0.11,0.00,0.11,0.00,6.8,2362,95619,10600,8238
6,2,61,17,1,True,2024-10-05 14:00:00+00:00,3.0,1.0,7,59,...,1,0.38,0.06,0.44,0.32,6.8,-1821,97477,6898,8719
7,2,71,3,1,False,2024-10-19 16:30:00+00:00,2.0,0.0,8,10,...,0,0.00,0.00,0.00,0.00,6.8,1340,103246,7657,6317
8,2,81,12,0,True,2024-10-27 16:30:00+00:00,2.0,2.0,9,5,...,0,0.18,0.00,0.18,0.00,6.8,-3526,101196,1944,5470
9,2,96,15,1,False,2024-11-02 12:30:00+00:00,1.0,0.0,10,19,...,0,0.00,0.01,0.01,0.04,6.8,-4233,98901,2012,6245


In [57]:
under = pb.scrapers.Understat("ENG Premier League", "2021-2022")
df = under.get_fixtures()
df

,understat_id,datetime,team_home,team_away,goals_home,goals_away,xg_home,xg_away,forecast_w,forecast_d,forecast_l,season,competition,date
id,,,,,,,,,,,,,,
1628812800---brentford---arsenal,16376,2021-08-13 19:00:00,Brentford,Arsenal,2,0,1.888180,1.023850,0.6289,0.2287,0.1424,2021-2022,ENG Premier League,2021-08-13
1628899200---burnley---brighton,16378,2021-08-14 14:00:00,Burnley,Brighton,1,2,1.795480,1.685300,0.3894,0.2877,0.3229,2021-2022,ENG Premier League,2021-08-14
1628899200---chelsea---crystal_palace,16379,2021-08-14 14:00:00,Chelsea,Crystal Palace,3,0,1.187090,0.321701,0.6405,0.2822,0.0773,2021-2022,ENG Premier League,2021-08-14
1628899200---everton---southampton,16380,2021-08-14 14:00:00,Everton,Southampton,3,1,2.388630,0.580601,0.8359,0.1234,0.0407,2021-2022,ENG Premier League,2021-08-14
1628899200---leicester---wolverhampton_wanderers,16381,2021-08-14 14:00:00,Leicester,Wolverhampton Wanderers,1,0,0.668082,1.327140,0.1683,0.2750,0.5567,2021-2022,ENG Premier League,2021-08-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1653177600---crystal_palace---manchester_united,16751,2022-05-22 15:00:00,Crystal Palace,Manchester United,1,0,0.354618,0.782005,0.1466,0.4028,0.4506,2021-2022,ENG Premier League,2022-05-22
1653177600---leicester---southampton,16752,2022-05-22 15:00:00,Leicester,Southampton,4,1,3.420580,1.051410,0.9057,0.0717,0.0226,2021-2022,ENG Premier League,2022-05-22
1653177600---liverpool---wolverhampton_wanderers,16753,2022-05-22 15:00:00,Liverpool,Wolverhampton Wanderers,3,1,3.193850,1.117080,0.8332,0.1119,0.0549,2021-2022,ENG Premier League,2022-05-22


In [60]:
player = under.get_player_season(8260)
player

,team,position,games,goals,shots,time,x_g,assists,x_a,key_passes,yellow,red,npg,npx_g,x_g_chain,x_g_buildup
season,,,,,,,,,,,,,,,,
2019,Borussia Dortmund,Sub,15,13,33,1061,8.889357395470142,2,1.5264755114912987,9,0,0,13,8.889357395470142,11.366187927313149,2.984985080547631
2020,Borussia Dortmund,FW,28,27,92,2416,23.59848389774561,6,4.03544756770134,27,2,0,25,20.567368783056736,27.197287127375603,5.896189399063587
2021,Borussia Dortmund,FW,24,22,80,1917,17.030953601002693,7,5.894002065062523,23,3,0,16,12.484372556209564,19.77244157716632,4.264734441414475
2022,Manchester City,FW,35,36,123,2803,32.761399537324905,8,5.8491098545491695,29,5,0,29,27.433209866285324,31.963398084044456,3.285816218703985
2023,Manchester City,FW,31,27,122,2581,31.65399668365717,5,4.7517555598169565,29,1,0,20,25.564646281301975,30.19725350290537,3.128645434975624
2024,Manchester City,FW,12,12,63,1080,13.286920607089996,0,0.8642479926347733,7,2,0,11,12.525751769542694,10.887216567993164,1.3429269380867481


## Expected Goals


In [29]:
# - Include Bookmarker odds
# - xG90 last 5 full matches
# - G90 last 5 full matches
# - Position
# - was home
# - FDR
# -----------------------------------------
# - Exclude Gks
# - Start with Gw 6
# - Only those who played 90 minutes are included
#

In [71]:
pd.read_csv('./data/joint/23-24/merged_extras_rolled_5/Bruno Borges Fernandes.csv')[['event','minutes', 'minutes_3', 'minutes_5']]

,event,minutes,minutes_3,minutes_5
0,1,90,0.0,0.0
1,2,90,90.0,90.0
2,3,90,180.0,90.0
3,4,90,270.0,180.0
4,5,90,270.0,270.0
5,6,90,270.0,360.0
6,7,90,270.0,360.0
7,8,90,270.0,450.0
8,9,90,270.0,450.0
9,10,90,270.0,450.0


In [51]:
pd.read_csv('./data/joint/23-24/merged_player_data.csv')['event'].min()

1

In [28]:
season = '23-24'
players_paths = next(os.walk('./data/joint/'+ season +'/merged_extras_xP', [None], [None],[]))[2]
for path in players_paths:
    player = pd.read_csv('./data/joint/'+ season +'/merged_extras_xP/'+ path, sep=',', skipinitialspace=True)
    player = player[player['minutes'] == 90]
    # print('===========>', player.columns)
    prev = [0, 0, 0, 0, 0]
    gwks = [1]

    def row_checking(row):
        # print('-------------->', prev)

        if(row['event'] - prev[-1] > 1):
            print()



        prev.append(row['event'])
        del prev[0]

        print('-------------->>>', prev)

    player.apply(row_checking, axis=1)

    # print(prev)


-------------->>> [0, 0, 0, 0, 28]
-------------->>> [0, 0, 0, 28, 38]
-------------->>> [0, 0, 0, 0, 3]
-------------->>> [0, 0, 0, 3, 8]
-------------->>> [0, 0, 0, 0, 1]
-------------->>> [0, 0, 0, 1, 2]
-------------->>> [0, 0, 1, 2, 3]
-------------->>> [0, 1, 2, 3, 4]
-------------->>> [1, 2, 3, 4, 13]
-------------->>> [2, 3, 4, 13, 28]
-------------->>> [0, 0, 0, 0, 1]
-------------->>> [0, 0, 0, 1, 3]
-------------->>> [0, 0, 1, 3, 4]
-------------->>> [0, 1, 3, 4, 11]
-------------->>> [1, 3, 4, 11, 18]
-------------->>> [3, 4, 11, 18, 19]
-------------->>> [4, 11, 18, 19, 21]
-------------->>> [11, 18, 19, 21, 30]
-------------->>> [18, 19, 21, 30, 31]
-------------->>> [19, 21, 30, 31, 32]
-------------->>> [21, 30, 31, 32, 33]
-------------->>> [30, 31, 32, 33, 34]
-------------->>> [31, 32, 33, 34, 35]
-------------->>> [32, 33, 34, 35, 36]
-------------->>> [33, 34, 35, 36, 37]
-------------->>> [34, 35, 36, 37, 38]
-------------->>> [0, 0, 0, 0, 1]
-------------->>> [0,

In [34]:
def add_rolling_avgs(season):
    print('================> starting season 20'+season)
    players_paths = next(os.walk('./data/joint/'+ season +'/merged_extras_xP', [None], [None],[]))[2]
    for path in players_paths:
        player = pd.read_csv('./data/joint/'+ season +'/merged_extras_xP/'+ path,sep=',', skipinitialspace=True)
        player = player[player['minutes'] == 90]
        print('===========>', player.columns)
        prev = [
                [0 for i in range(34)],
                [0 for i in range(34)],
                [0 for i in range(34)],
                [0 for i in range(34)]
            ]
        gwks = [1]


        ['assists_x', 'bonus', 'bps', 'clean_sheets', 'creativity', 'element',
       'expected_assists', 'expected_goal_involvements', 'expected_goals',
       'expected_goals_conceded', 'fixture', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'date', 'minutes', 'opponent_team',
       'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards',
       'round', 'saves', 'selected', 'starts', 'team_a_score', 'team_h_score',
       'threat', 'total_points', 'transfers_balance', 'transfers_in',
       'transfers_out', 'value', 'was_home', 'yellow_cards', 'goals', 'shots',
       'xG', 'time', 'h_team', 'a_team', 'h_goals', 'a_goals', 'id', 'season',
       'roster_id', 'xA', 'assists_y', 'key_passes', 'npg', 'npxG', 'xGChain',
       'xGBuildup', 'team_h_difficulty', 'team_a_difficulty', 'event', 'xP',
       'position'],

        def rolling(row):
            row_items = [
                row['clean_sheets'], row['expected_assists'], row['expected_goal_involvements'], row['expected_goals'], row['expected_goals_conceded'],
                row['goals_conceded'], row['goals_scored'], row['ict_index'], row['influence'], row['creativity'], row['threat'], row['minutes'], row['own_goals'], row['penalties_missed'],
                row['penalties_saved'], row['red_cards'], row['yellow_cards'], row['saves'], row['starts'],  row['team_a_score'],  row['team_h_score'], row['total_points'],  row['goals'],
                row['shots'], row['xG'], row['xA'], row['assists_y'], row['key_passes'], row['npg'], row['npxG'], row['xGChain'],  row['xGBuildup'], row['xP'], row['selected']
                ]

            if row['event'] - gwks[0] == 0:
                del prev[3]
                prev.append(row_items)

            elif row['event'] - gwks[0] == 1:
                del prev[0]
                prev.append(row_items)
                gwks[0] = row['event']

            elif row['event'] - gwks[0] == 2:
                del prev[0]
                del prev[0]
                prev.append([0 for i in range(34)])
                prev.append(row_items)
                gwks[0] = row['event']

            else:
                del prev[0]
                del prev[0]
                del prev[0]
                del prev[0]
                prev.append([0 for i in range(34)])
                prev.append([0 for i in range(34)])
                prev.append([0 for i in range(34)])
                prev.append(row_items)
                gwks[0] = row['event']

            # print(row['event'], len(pd.Series([round((x+y+z)/3 ,2) for x,y,z in zip(prev[0], prev[1], prev[2])])),  prev[0], prev[1], prev[2], prev[3]) #
            return pd.Series([round((x+y+z) ,2) for x,y,z in zip(prev[0], prev[1], prev[2])])
        # print(gwks, player.apply(rolling, axis=1).shape)
        player[
                [
                    'clean_sheets_3', 'expected_assists_3', 'expected_goal_involvements_3', 'expected_goals_3', 'expected_goals_conceded_3',
                    'goals_conceded_3', 'goals_scored_3', 'ict_index_3', 'influence_3', 'creativity_3', 'threat_3', 'minutes_3', 'own_goals_3', 'penalties_missed_3', 'penalties_saved_3',
                    'red_cards_3', 'yellow_cards_3', 'saves_3', 'starts_3',  'team_a_score_3',  'team_h_score_3', 'total_points_3', 'goals_3', 'shots_3', 'xG_3',
                    'xA_3', 'assists_y_3', 'key_passes_3', 'npg_3', 'npxG_3', 'xGChain_3',  'xGBuildup_3',  'xP_3', 'selected_3'
                ]
            ] = player.apply(rolling, axis=1)

        # Save the updated DataFrame with the new columns
        new_col_dir = './data/joint/'+ season +'/merged_extras_rolled/'
        if not os.path.exists(new_col_dir):
            os.makedirs(new_col_dir)
        player.to_csv(new_col_dir + path, index=False)

    print('<<<<================ starting season 20'+season)

In [36]:
# add_rolling_avgs('21-22')

add_rolling_avgs('22-23')
add_rolling_avgs('23-24')

================> starting season 2022-23
===========> Index(['assists_x', 'bonus', 'bps', 'clean_sheets', 'creativity', 'element',
       'expected_assists', 'expected_goal_involvements', 'expected_goals',
       'expected_goals_conceded', 'fixture', 'goals_conceded', 'goals_scored',
       'ict_index', 'influence', 'date', 'minutes', 'opponent_team',
       'own_goals', 'penalties_missed', 'penalties_saved', 'red_cards',
       'round', 'saves', 'selected', 'starts', 'team_a_score', 'team_h_score',
       'threat', 'total_points', 'transfers_balance', 'transfers_in',
       'transfers_out', 'value', 'was_home', 'yellow_cards', 'goals', 'shots',
       'xG', 'time', 'h_team', 'a_team', 'h_goals', 'a_goals', 'id', 'season',
       'roster_id', 'xA', 'assists_y', 'key_passes', 'npg', 'npxG', 'xGChain',
       'xGBuildup', 'team_h_difficulty', 'team_a_difficulty', 'event', 'xP',
       'position'],
      dtype='object')
===========> Index(['assists_x', 'bonus', 'bps', 'clean_sheets', 'cr

ValueError: Columns must be same length as key